[Back to the starting page](start.ipynb)

In [ ]:
from scripts import *
import inspect
from sklearn import linear_model
#
# THIS NOTEBOOK CANNOT BE RUN ON BINDER => limitation 2 Gb RAM !!
#

In [ ]:
# Load data (lon,lat)
x,y,elev                   = load_etopo(corrected_from_ice=True)
x,y,crustal_thickness      = load_crustal_thickness()
x,y,strain_rate            = load_strain_rate()
x,y,dist_closest_hs        = load_hotspots()
x,y,age                    = load_seafloor_ages()
x,y,lithospheric_thickness = load_lithospheric_thickness()
x,y,age_lithos             = load_age_lithos()

# Weights: to correct for area variations with latitude
xlon, ylat = np.meshgrid(x,y)
W = 6371*np.cos(np.abs(ylat)*np.pi/180)*2*np.pi/(360/(x[1]-x[0]))

# correction for water load
elev = np.where(elev<0,elev-elev*1030/3400,elev)

min_dist_hs           = 1000e3 # m
threshold_strain_rate = 1e-16  # 1/s
ref_crustal_thickness = 35     # km

In [ ]:
# Plot continental datasets
#
# plt.show/plt.pcolormesh is slow
#

%matplotlib widget

# Arguments description
print(inspect.getdoc(plot_global_maps_continents))

# Select data
# In case you want to select as a function of lon and lat, you must use xlon, ylat

# Stable continents far from hotspots with 35±5 km thick crust
mask = ( (~age.mask) | 
         (crustal_thickness<ref_crustal_thickness-5) | 
         (crustal_thickness>ref_crustal_thickness+5) |
         (strain_rate>threshold_strain_rate)  | 
         (dist_closest_hs < min_dist_hs) )

# Stable continents far from hotspots
#mask = ( (~age.mask) | 
#         (strain_rate>threshold_strain_rate)  | 
#         (dist_closest_hs < min_dist_hs) )

# All continents
#mask = (~age.mask)

data = np.ma.masked_array(elev, mask)

plot_figure = False

if plot_figure:
    plot_global_maps_continents(x                     = x,
                                y                     = y,
                                data                  = data,
                                typedat               = 'elevation', # to set the display (xlim,ticks...)
                                plotcircles_hs        = True,
                                orientationbar        = 'vertical',
                                savefig               = False)

In [ ]:
# Elevation distribution of continents

%matplotlib widget

# mask and data from previous cell

Wm   = np.ma.masked_array(W, mask)

plot_histo(data[~data.mask].reshape(-1),
           'Elevation',
           weights = Wm[~Wm.mask].reshape(-1),
           xlabel='Elevation (m)',
           unit='m',
           GaussianModel=True,
           sigmamodel=200,
           meanmodel=450,
           text="right",
           xlim=[-1000,2500],
           savefig=False)


In [ ]:
# Elevation distribution for stable continents with crustal thickness 35 +/- 5 km far from hotspots 
# +/- Antarctic

%matplotlib widget

# Excluding Antarctic
mask = ( (~age.mask) | 
         (crustal_thickness<ref_crustal_thickness-5) | 
         (crustal_thickness>ref_crustal_thickness+5) |
         (strain_rate>threshold_strain_rate)  | 
         (dist_closest_hs < min_dist_hs) |
         (ylat<-60) )
data = np.ma.masked_array(elev, mask)
Wm   = np.ma.masked_array(W, mask)

plot_histo(data[~data.mask].reshape(-1),
           'Elevation',
           weights = Wm[~Wm.mask].reshape(-1),
           xlabel='Elevation (m)',
           unit='m',
           GaussianModel=True,
           sigmamodel=175,
           meanmodel=200,
           text="right",
           xlim=[-1000,2500],
           savefig=False,
           filename='histo1.pdf')

# Only Antarctic
mask = ( (~age.mask) | 
         (crustal_thickness<ref_crustal_thickness-5) | 
         (crustal_thickness>ref_crustal_thickness+5) |
         (strain_rate>threshold_strain_rate)  | 
         (dist_closest_hs < min_dist_hs) |
         (ylat>=-60) )
data = np.ma.masked_array(elev, mask)
Wm   = np.ma.masked_array(W, mask)

plot_histo(data[~data.mask].reshape(-1),
           'Elevation',
           weights = Wm[~Wm.mask].reshape(-1),
           xlabel='Elevation (m)',
           unit='m',
           GaussianModel=False,
           text="right",
           xlim=[-1000,2500],
           savefig=False,
           filename='histo2.pdf')


In [ ]:
# Crustal thickness distribution for stable continents far from hotspots
%matplotlib widget

mask = ( (~age.mask) | 
         (strain_rate>threshold_strain_rate)  | 
         (dist_closest_hs < min_dist_hs) )
data = np.ma.masked_array(crustal_thickness, mask)
Wm   = np.ma.masked_array(W, mask)

plot_histo(data[~data.mask].reshape(-1),
           'Crustal thickness',
           weights = Wm[~Wm.mask].reshape(-1),
           xlabel='Thickness (km)',
           unit='km',
           GaussianModel=True,
           sigmamodel=5,
           meanmodel=35,
           text="right",
           xlim=[5,75],
           savefig=False)


In [ ]:
# Correlation Elevation versus crustal thickness for stable continents far from hotspots

mask          = ((age.mask) & (strain_rate<1e-16) & (dist_closest_hs > 1e6))
mask_default  = (age.mask)
Wm            = W[mask_default]
count_default = np.count_nonzero(Wm)
Wm            = W[mask]
count         = np.count_nonzero(Wm)
print('Stable continents far from hotspots represent {} % of the continental domain '.format(count*100/count_default))

y        = elev[mask]
x        = crustal_thickness[mask]

%matplotlib widget

title = 'elev continents vs crustal thickness'
plot_correlation(x,y,title,
                 nbins = 20,
                 weights=Wm,
                 ylabel='Elevation (m)',
                 xlabel='Crustal thickness (km)',
                 xlim=[15,65],
                 ylim=[-1500,4000],
                 ticks=[10,5,2000,250],
                 text="left",
                 plottext=False,
                 savefig=False)

In [ ]:
regr = linear_model.LinearRegression()
regr.fit(x.reshape(-1, 1), y,Wm)
xtest = np.linspace(15,65,100)
ypred = regr.predict(xtest.reshape(-1, 1))
plt.plot(xtest,ypred,label='Linear fit')
plt.legend()
#plt.savefig('correlation.pdf',dpi=300)
plt.show()

In [ ]:
# Correlation Elevation versus continental lithospheric thickness 
# for stable continents with crustal thickness 35 +/- 5 km far from hotspots

mask          = ((age.mask) & (crustal_thickness>30) 
                            & (crustal_thickness<40) 
                            & (strain_rate<1e-16)
                            & (dist_closest_hs > 1e6))
mask_default  = (age.mask)
Wm            = W[mask_default]
count_default = np.count_nonzero(Wm)
Wm            = W[mask]
count         = np.count_nonzero(Wm)
print('Stable continents with crustal thickness 35 +/- 5 km far from hotspots represent {} % of the continental domain '.format(count*100/count_default))

y        = elev[mask]
x        = lithospheric_thickness[mask]

%matplotlib widget
title = 'elev continents vs lithospheric thickness'
plot_correlation(x,y,title,
                 nbins = 20,
                 weights=Wm,
                 ylabel='Elevation (m)',
                 xlabel='Lithospheric thickness (km)',
                 xlim=[0,250],
                 ylim=[-1000,2500],
                 ticks=[50,25,2000,250],
                 text="left",
                 plottext=False,
                 savefig=False,
                 fig_y=6.4)

In [ ]:
# Correlation Elevation versus continental lithospheric age 
# for stable continents with crustal thickness 35 +/- 5 km far from hotspots

mask          = ((age.mask) & (crustal_thickness>30) 
                            & (crustal_thickness<40) 
                            & (strain_rate<1e-16)
                            & (dist_closest_hs > 1e6))
mask_default  = (age.mask)
Wm            = W[mask_default]
count_default = np.count_nonzero(Wm)
Wm            = W[mask]
count         = np.count_nonzero(Wm)
print('Stable continents with crustal thickness 35 +/- 5 km far from hotspots represent {} % of the continental domain '.format(count*100/count_default))

y        = elev[mask]
x        = age_lithos[mask]

%matplotlib widget
title = 'elev continents vs lithospheric thickness'
plot_correlation(x,y,title,
                 nbins = 20,
                 weights=Wm,
                 ylabel='Elevation (m)',
                 xlabel='Lithospheric age (Ma)',
                 xlim=[0,4500],
                 ylim=[-1000,2500],
                 ticks=[500,250,2000,250],
                 text="left",
                 plottext=False,
                 savefig=False,
                 fig_y=6.4)